### GPT for Learning how to play 2048

Inspired by [GPT from scratch](https://www.youtube.com/watch?v=kCc8FmEb1nY)

#### Explore the data

In [1]:
with open('games.txt', 'r') as f:
    text = f.read()

print(text[:1000])
print(len(text))

00000201000000000000000000000000,00000000000000000000000000010201,00010201000000000000000000000001,00010202010000000000000000000000,01030000010000010000000000000000,02030001000000000000000000010000,02030001000100000000000001000000,02030001010100000000000000010000,02030001010200000000000000020000,02030001010300000000000001000000,02040001020000000000010000000000,03040101000000000000000000000100,03040201000000000000000000010000,03040201010100000000000000000000,03040201020000000002000000000000,03040201020200010000000000000000,03040202020200010000000000000000,03040300030100000001000000000000,04040300000200000000000000020000,00000000000000000004010004030300,00000000000000000000040101000404,00000000000000010000040100000105,00000000000000000001040200000105,00000000000000000001040200010105,00000000010000000001040200000205,00000100000000000000040201010205,00010001000000000000040200020205,00000002010000000000040200000305,00000100000000000000040301000305,00000000000001000100040301000305,0000010000

#### Tokenize

Each tile is represented by 2 digit, 0-led int. Will convert tile into regular int. 

Each state is separated by comma, each game by ';\n'. Will convert these into int as well

In [2]:
mapping = {',':17, ';\n':18}
inv_mapping = {v: k for k, v in mapping.items()}

def encode(s:str)-> list[str]:
    out = []
    num_splits = len(s.split(f';\n'))
    for i, game in enumerate(s.split(f';\n')): 
        for state in game.split(',')[:-1]:
            enc_state = [int(''.join(state[i:i+2])) for i in range(0, len(state), 2)]
            enc_state.append(mapping[','])
            out += enc_state
        out.append(mapping[';\n']) if i < num_splits-1 else None
    
    return out

def decode(l:list[int]) -> str:
    s = ''
    for char in l:
        if char < 10:
            s += '0'+ str(char)
        elif char < min(mapping.values()): # hard coded, should fix
            s += str(char)
        else:
            s += inv_mapping[char]

    return s        


print(encode('02000000020303010607060507081011,01000000030303010607060507081011,;\n01000000030303010607060507081011,'))
print(decode(encode('02000000020303010607060507081011,01000000030303010607060507081011,;\n01000000030303010607060507081011,')))


[2, 0, 0, 0, 2, 3, 3, 1, 6, 7, 6, 5, 7, 8, 10, 11, 17, 1, 0, 0, 0, 3, 3, 3, 1, 6, 7, 6, 5, 7, 8, 10, 11, 17, 18, 1, 0, 0, 0, 3, 3, 3, 1, 6, 7, 6, 5, 7, 8, 10, 11, 17]
02000000020303010607060507081011,01000000030303010607060507081011,;
01000000030303010607060507081011,


#### Load data

Get train test split

Set up batches

In [3]:
import torch

torch.manual_seed(1748)

data = torch.tensor(encode(text), dtype=torch.long)

batch_size = 4
block_size = 34 # two boards
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 100
num_iters = 1000
eval_interval = 100
n_embed = 32
print(device)

# get first game in last 20% of data
n = int(0.8*len(data))
while(data[n] != mapping[';\n']):
    n += 1
n += 2

train_data = data[:n]
test_data = data[n:]
print(n, len(data), len(train_data), len(test_data))

def get_batch(split:bool=0)-> list[torch.Tensor]:
    # split == 0: train, 1: test
    data = train_data if split == 0 else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x = x.to(device)
    y = y.to(device)
    return x, y

x, y = get_batch()

print(x)

print(y)



cuda
1816089 2223064 1816089 406975
tensor([[ 2,  0,  1,  0,  0,  3,  1,  0,  1,  2,  6,  2,  3,  5,  3, 17,  2,  1,
          0,  1,  3,  1,  0,  0,  1,  2,  6,  0,  2,  3,  5,  3, 17,  0],
        [ 0,  0,  0,  1,  0,  0,  4,  2,  1,  0,  2,  2, 17,  0,  0,  1,  0,  0,
          0,  0,  0,  2,  0,  4,  1,  1,  0,  2,  3, 17,  0,  0,  0,  1],
        [ 2,  4,  7,  8, 12, 17,  2,  2,  0,  0,  1,  2,  0,  0,  1,  5,  4,  2,
          4,  7,  8, 12, 17,  0,  1,  0,  3,  0,  0,  1,  2,  1,  5,  4],
        [ 6,  2,  4, 10, 11, 17,  1,  1,  4,  2,  3,  4,  5,  4,  7,  8,  7,  6,
          2,  4, 10, 11, 17,  2,  4,  2,  1,  3,  4,  5,  4,  7,  8,  7]],
       device='cuda:0')
tensor([[ 0,  1,  0,  0,  3,  1,  0,  1,  2,  6,  2,  3,  5,  3, 17,  2,  1,  0,
          1,  3,  1,  0,  0,  1,  2,  6,  0,  2,  3,  5,  3, 17,  0,  0],
        [ 0,  0,  1,  0,  0,  4,  2,  1,  0,  2,  2, 17,  0,  0,  1,  0,  0,  0,
          0,  0,  2,  0,  4,  1,  1,  0,  2,  3, 17,  0,  0,  0,  1,  1],
        [

#### Bigram Model

Loss 

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1748)

class Head(nn.module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)

        wei = q @ k.transpose(-2,-1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)

        v = self.value(x)
        out = wei @ v
        return out

class BigramModel(nn.Module):

    def __init__(self, vocab_size=19):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tok_emb + pos_emb
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B,T)
        for  _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:,-1,:] # last time step, (B,C)
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            idx = torch.cat((idx, idx_next), dim=1) #(B, T+1)
        return idx

    
model = BigramModel()
m = model.to(device)
logits, loss = m(x,y)
print(logits.shape)
print(loss)

# print(decode(m.generate(torch.zeros((1,1), dtype=torch.long, device=device), max_new_tokens=100)[0].tolist()))

torch.Size([136, 19])
tensor(3.2514, device='cuda:0', grad_fn=<NllLossBackward0>)


In [5]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in [0, 1]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)



for iter in range(num_iters):

    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f'{iter}: Train loss: {losses[0]:.4f}, Val loss: {losses[1]:.4f}')
    xb, yb = get_batch(0)

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# print(decode(m.generate(torch.zeros((1,1), dtype=torch.long, device=device), max_new_tokens=100)[0].tolist()))



0: Train loss: 3.2024, Val loss: 3.1715
100: Train loss: 2.2965, Val loss: 2.2842
200: Train loss: 2.0542, Val loss: 2.0276
300: Train loss: 1.9668, Val loss: 1.9174
400: Train loss: 1.9273, Val loss: 1.8390
500: Train loss: 1.8722, Val loss: 1.8168
600: Train loss: 1.8810, Val loss: 1.8089
700: Train loss: 1.8484, Val loss: 1.7669
800: Train loss: 1.8215, Val loss: 1.7519
900: Train loss: 1.8461, Val loss: 1.7428
